In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from seaborn import set_style
import kagglehub
set_style("whitegrid")

In [3]:
path = kagglehub.dataset_download("ehallmar/daily-historical-stock-prices-1970-2018")

print("Path to dataset files:", path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469M/469M [00:17<00:00, 27.9MB/s]

Extracting model files...


Path to dataset files: /home/hava/.cache/kagglehub/datasets/ehallmar/daily-historical-stock-prices-1970-2018/versions/1


In [5]:
prices_file = path + '/historical_stock_prices.csv'
labels_file = path + '/historical_stocks.csv'
df_prices = pd.read_csv(prices_file)
df_labels = pd.read_csv(labels_file)
df_prices

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


In [6]:
finance = df_labels[df_labels['sector']=='FINANCE'][['ticker','name']]
finance['ticker']=finance['ticker'].astype(str)
print(finance['ticker'])
df_finance = df_prices[df_prices['ticker'].isin(finance['ticker'])].reset_index()
df_finance

0         PIH
1       PIHPP
2        TURN
4        FCCY
5        SRCE
        ...  
6409      WTM
6421       WF
6436       XL
6444      YRD
6454      ZBK
Name: ticker, Length: 1022, dtype: object


,index,ticker,open,close,adj_close,low,high,volume,date
0,0,AHH,11.500000,11.580000,8.493155,11.250000,11.680000,4633900,2013-05-08
1,1,AHH,11.660000,11.550000,8.471151,11.500000,11.660000,275800,2013-05-09
2,2,AHH,11.550000,11.600000,8.507822,11.500000,11.600000,277100,2013-05-10
3,3,AHH,11.630000,11.650000,8.544494,11.550000,11.650000,147400,2013-05-13
4,4,AHH,11.600000,11.530000,8.456484,11.500000,11.600000,184100,2013-05-14
...,...,...,...,...,...,...,...,...,...
3134206,20896678,SPGI,203.399994,203.919998,203.919998,203.179993,204.449997,586000,2018-08-20
3134207,20896680,SPGI,204.229996,204.350006,204.350006,203.860001,205.320007,1311200,2018-08-21
3134208,20896682,SPGI,203.869995,202.919998,202.919998,202.839996,204.809998,696700,2018-08-22
3134209,20896684,SPGI,202.990005,202.850006,202.850006,202.589996,204.639999,759500,2018-08-23


In [7]:
df_finance['date'] = pd.to_datetime(df_finance['date'])
df_finance['ticker'] = (df_finance['ticker']).astype(str)
dst_dates_df = pd.read_csv('DaylightSavingsTimeChangeDates_1971-2024.csv')
dst_dates_spring = pd.to_datetime(dst_dates_df.Date+" "+dst_dates_df.Year.astype(str))
dst_dates_fall = pd.to_datetime(dst_dates_df['Date.1']+" "+dst_dates_df.Year.astype(str))
dst_dates = pd.concat([dst_dates_fall,dst_dates_spring])
dst_mon = dst_dates+pd.DateOffset(1)
dst_fri = dst_dates+pd.DateOffset(-2)
dst_fall_mon = dst_dates_fall+pd.DateOffset(1)
dst_fall_fri = dst_dates_fall+pd.DateOffset(-2)
dst_spring_mon = dst_dates_spring+pd.DateOffset(1)
dst_spring_fri = dst_dates_spring+pd.DateOffset(-2)

In [11]:
finance_dst_mon_fall = df_finance[df_finance.date.isin(dst_fall_mon)]
finance_dst_mon_fall = finance_dst_mon_fall.assign(Season='Fall')
finance_dst_mon_spring = df_finance[df_finance.date.isin(dst_spring_mon)]
finance_dst_mon_spring = finance_dst_mon_spring.assign(Season='Spring')
finance_dst_mon = pd.concat([finance_dst_mon_fall,finance_dst_mon_spring])
finance_dst_fri = df_finance[df_finance.date.isin(dst_fri)]

for tkr in finance['ticker'].astype(str):
    mon_tkr = finance_dst_mon[finance_dst_mon.ticker.astype(str)==tkr]
    fri_tkr = finance_dst_fri[finance_dst_fri.ticker.astype(str)==tkr]
    finance_dst_mon = finance_dst_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    finance_dst_fri = finance_dst_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
    #finance_dst_mon = finance_dst_mon.drop(finance_dst_mon[(finance_dst_mon.ticker.astype(str)==tkr  finance_dst_mon['date'].isin(finance_dst_fri[finance_dst_fri.ticker.astype(str)==tkr].date+pd.DateOffset(3))==False)].index)
    #finance_dst_fri = finance_dst_fri.drop(finance_dst_fri[(finance_dst_fri['ticker'==tkr] & finance_dst_fri['date'].isin(finance_dst_mon['ticker'==tkr].date+pd.DateOffset(-3))==False)].index)
print(len(finance_dst_mon))
print(len(finance_dst_fri))

22807
22807


In [12]:
finance_dst_fri = finance_dst_fri.reset_index()
finance_dst_mon = finance_dst_mon.reset_index()

In [13]:
finance_dst = pd.DataFrame()
finance_dst['ticker'] = finance_dst_mon['ticker']
finance_dst['mon_date'] = finance_dst_mon['date']
finance_dst['mon_open'] = finance_dst_mon['open']
finance_dst['mon_close'] = finance_dst_mon['close']
finance_dst['mon_adj_close'] = finance_dst_mon['adj_close']
finance_dst['mon_low'] = finance_dst_mon['low']
finance_dst['mon_high'] = finance_dst_mon['high']
finance_dst['mon_volume'] = finance_dst_mon['volume']
finance_dst['fri_date'] = finance_dst_fri['date']
finance_dst['fri_open'] = finance_dst_fri['open']
finance_dst['fri_close'] = finance_dst_fri['close']
finance_dst['fri_adj_close'] = finance_dst_fri['adj_close']
finance_dst['fri_low'] = finance_dst_fri['low']
finance_dst['fri_high'] = finance_dst_fri['high']
finance_dst['fri_volume'] = finance_dst_fri['volume']
finance_dst['Season'] = finance_dst_mon['Season']
finance_dst

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume,Season
0,AHH,2013-11-04,9.660000,9.620000,7.214474,9.560000,9.700000,49000,2013-11-01,9.600000,9.620000,7.214474,9.520000,9.650000,77900,Fall
1,APO,2011-11-07,13.190000,13.910000,9.635016,13.190000,13.950000,354900,2014-03-07,10.130000,10.050000,7.666063,9.900000,10.300000,81300,Fall
2,AHH,2014-11-03,9.420000,9.380000,7.520571,9.360000,9.520000,138400,2011-11-04,12.970000,13.220000,9.157074,12.910000,13.250000,128000,Fall
3,APO,2012-11-05,14.920000,14.810000,10.486925,14.640000,15.070000,184000,2014-10-31,9.440000,9.380000,7.520571,9.370000,9.580000,256000,Fall
4,AHH,2015-11-02,10.810000,10.860000,9.308818,10.720000,10.930000,125700,2012-03-09,14.100000,14.220000,9.957026,13.960000,14.250000,119600,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22802,SPGI,2014-03-10,81.529999,82.040001,77.746552,81.220001,82.290001,1437500,2016-03-11,94.000000,95.129997,92.710060,93.419998,95.220001,1252300,Spring
22803,SPGI,2015-03-09,101.430000,103.470001,99.439667,101.239998,103.730003,1978900,2016-11-04,122.480003,122.589996,120.224899,121.919998,123.610001,1511400,Spring
22804,SPGI,2016-03-14,94.410004,93.839996,91.452873,93.760002,94.940002,1652600,2017-03-10,131.500000,131.690002,129.942780,130.649994,132.080002,1166300,Spring
22805,SPGI,2017-03-13,132.000000,132.589996,130.830826,131.949997,132.949997,871900,2017-11-03,156.240005,158.690002,157.477264,154.699997,158.800003,1755500,Spring


In [14]:
finance_mon = df_finance[df_finance.date.dt.day_name()=='Monday']
finance_mon = finance_mon.drop(finance_mon[finance_mon.date.isin(dst_mon)].index)
finance_fri = df_finance[df_finance.date.dt.day_name()=='Friday']
finance_fri = finance_fri.drop(finance_fri[finance_fri.date.isin(dst_fri)].index)
for tkr in finance['ticker'].astype(str):
    mon_tkr = finance_mon[finance_mon.ticker.astype(str)==tkr]
    fri_tkr = finance_fri[finance_fri.ticker.astype(str)==tkr]
    finance_mon = finance_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    finance_fri = finance_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
print(len(finance_mon))
print(len(finance_fri))

516370
516370


In [15]:
finance_mon = finance_mon.reset_index()
finance_fri = finance_fri.reset_index()

In [16]:
finance_wkd = pd.DataFrame()
finance_wkd['ticker'] = finance_mon['ticker']
finance_wkd['mon_date'] = finance_mon['date']
finance_wkd['mon_open'] = finance_mon['open']
finance_wkd['mon_close'] = finance_mon['close']
finance_wkd['mon_adj_close'] = finance_mon['adj_close']
finance_wkd['mon_low'] = finance_mon['low']
finance_wkd['mon_high'] = finance_mon['high']
finance_wkd['mon_volume'] = finance_mon['volume']
finance_wkd['fri_date'] = finance_fri['date']
finance_wkd['fri_open'] = finance_fri['open']
finance_wkd['fri_close'] = finance_fri['close']
finance_wkd['fri_adj_close'] = finance_fri['adj_close']
finance_wkd['fri_low'] = finance_fri['low']
finance_wkd['fri_high'] = finance_fri['high']
finance_wkd['fri_volume'] = finance_fri['volume']
finance_wkd

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume
0,AHH,2013-05-13,11.630000,11.650000,8.544494,11.550000,11.650000,147400,2013-05-10,11.550000,11.600000,8.507822,11.500000,11.600000,277100
1,AHH,2013-05-20,11.760000,11.730000,8.603169,11.630000,11.830000,46800,2013-05-17,11.700000,11.760000,8.625172,11.700000,11.850000,305400
2,AHH,2013-06-03,11.680000,11.520000,8.449149,11.300000,11.680000,143200,2013-05-31,11.600000,11.470000,8.412476,11.470000,11.800000,109400
3,AHH,2013-06-10,11.600000,11.650000,8.544494,11.500000,11.670000,167200,2013-06-07,11.670000,11.600000,8.507822,11.520000,11.680000,52300
4,AHH,2013-06-17,11.480000,11.550000,8.471151,11.400000,11.650000,73300,2013-06-14,11.490000,11.450000,8.397809,11.300000,11.490000,73100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516365,SPGI,2018-07-23,212.899994,212.869995,212.869995,212.300003,213.669998,790500,2018-07-20,210.699997,212.330002,212.330002,209.509995,212.789993,882000
516366,SPGI,2018-07-30,203.410004,200.880005,200.880005,200.289993,204.809998,1484300,2018-07-27,212.580002,203.860001,203.860001,202.490005,213.529999,1848400
516367,SPGI,2018-08-06,198.000000,198.529999,198.529999,197.690002,198.940002,1013900,2018-08-03,198.070007,198.080002,198.080002,196.210007,198.149994,1032600
516368,SPGI,2018-08-13,201.309998,200.529999,200.529999,200.389999,202.220001,949200,2018-08-10,200.460007,201.320007,201.320007,199.750000,201.929993,843700


In [17]:
finance_dst.to_csv('DST_Finance_Stocks.csv')
finance_wkd.to_csv('Weekends_Finance_Stocks.csv')